# Format Topographic Features

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-01

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Format Topographic Features
# Author: Timm Nawrocki
# Created on: 2019-04-01
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Format Topographic Features" extracts the topographic features to the area of interest with standardized grid and cell size.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import time

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'F:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')
elevation_source = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
area_of_interest = os.path.join(root_directory, 'Data_Input/area_of_interest/area_of_interest.tif')
elevation = os.path.join(elevation_source, 'Alaska_USGS3DEP_Elevation_60m_AKALB_20181106.tif')
compound_topographic = os.path.join(elevation_source, 'Alaska_Topography_CompoundTopographic_60m_AKALB_20181106.tif')
heat_load = os.path.join(elevation_source, 'Alaska_Topography_HeatLoad_60m_AKALB_20181106.tif')
integrated_moisture = os.path.join(elevation_source, 'Alaska_Topography_IntegratedMoisture_60m_AKALB_20181106.tif')
linear_aspect = os.path.join(elevation_source, 'Alaska_Topography_LinearAspect_60m_AKALB_20181106.tif')
mean_slope = os.path.join(elevation_source, 'Alaska_Topography_MeanSlope3x3_60m_AKALB_20181106.tif')
roughness = os.path.join(elevation_source, 'Alaska_Topography_Roughness3x3_60m_AKALB_20181106.tif')
site_exposure = os.path.join(elevation_source, 'Alaska_Topography_SiteExposure_60m_AKALB_20181106.tif')
surface_area_ratio = os.path.join(elevation_source, 'Alaska_Topography_SurfaceAreaRatio_60m_AKALB_20181106.tif')
surface_relief = os.path.join(elevation_source, 'Alaska_Topography_SurfaceRelief3x3_60m_AKALB_20181106.tif')

# Define output file names
output_names = ['elevation.tif',
                'compoundTopographic.tif',
                'heatLoad.tif',
                'moisture.tif',
                'aspect.tif',
                'slope.tif',
                'roughness.tif',
                'exposure.tif',
                'surfaceArea.tif',
                'surfaceRelief.tif'
               ]

# Define output path
output_path = os.path.join(root_directory, 'Data_Input/predictor_env')

# Define output files
output_array = []
for name in output_names:
    file = os.path.join(output_path, name)
    output_array = output_array + [file]

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Format Topographic Features

In [4]:
# Define a function to format the topographic features to standard grid and cell size
def format_topographic(**kwargs):
    """
    Description: extracts topographic rasters to the area of interest raster with grid and cell size matched to the study area raster
    Inputs: 'input_array' -- an array containing the input rasters with the first raster as the area of interest and any number of additional topographic rasters
            'output_array' -- an array of one output name per input raster
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_rasters = kwargs['input_array']
    area_of_interest = input_rasters.pop(0)
    output_array = kwargs['output_array']
    # Set the snap raster and cell size environments
    arcpy.env.snapRaster = area_of_interest
    arcpy.env.cellSize = area_of_interest
    # Define a function to format predictor rasters
    def formatRaster(in_raster, extract_mask, out_raster):
        # Extract projected raster to the area of interest
        print('Extracting raster to area of interest...')
        extract_raster = ExtractByMask(in_raster, extract_mask)
        # Copy results to outRaster
        print('Preparing output raster...')
        arcpy.CopyRaster_management(extract_raster,
                                    out_raster,
                                    '',
                                    '',
                                    '-32768',
                                    'NONE',
                                    'NONE',
                                    '32_BIT_FLOAT',
                                    'NONE',
                                    'NONE',
                                    'TIFF',
                                    'NONE')
    # Iterate the format raster function for each raster selected as an input
    count = 0
    for input_raster in input_rasters:
        output_raster = output_array[count]
        formatRaster(input_raster, area_of_interest, output_raster)
        count += 1
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully formatted topographic features...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [5]:
# Define input and output arrays
format_topographic_inputs = [area_of_interest,
                             elevation,
                             compound_topographic,
                             heat_load,
                             integrated_moisture,
                             linear_aspect,
                             mean_slope,
                             roughness,
                             site_exposure,
                             surface_area_ratio,
                             surface_relief
                            ]
format_topographic_outputs = output_array

# Create key word arguments
format_topographic_kwargs = {'input_array' : format_topographic_inputs,
                             'output_array' : format_topographic_outputs
                            }

# Process the create polygon function with the point array
arcpy_geoprocessing(format_topographic, **format_topographic_kwargs)

Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Extracting raster to area of interest...
Preparing output raster...
Successfully formatted topographic features...
----------
Succeeded at 2019-04-02 20:42 (Elapsed time: 2:14:13)
